In [1]:
from etl.src.bigquery_operators import RunQueryAndSaveToNewTable

In [3]:
exporter = RunQueryAndSaveToNewTable(
    project="codersrank",
    create_table_sql="./sql/create_new_table_for_extracted_data.sql",
    sql="./sql/extract_data_from_gh_archive_old_api.sql",
    gh_archive_dataset="day",
    gh_archive_table="20130410",
    dataset="project_looc_raw",
    dest_table="raw_export_test",
    dataset_location="US"
)

In [4]:
exporter.execute_dry_run()

2020-10-11 20:33:05,335 INFO COLLECTING DATA: WITH
  export_commit_data AS(
  SELECT
    event_id,
    event_type,
    created_at,
    repo_name,
    repo_id,
    repo_url,
    actor_name,
    push_id,
    commit_data
  FROM (
    SELECT
      *,
      ARRAY(
      SELECT
        JSON_EXTRACT(x,
          '$')
      FROM
        UNNEST(array_commits) x) AS commit_data
    FROM (
      SELECT
        id AS event_id,
        type AS event_type,
        created_at,
        repo.name AS repo_name,
        repo.id AS repo_id,
        repo.url AS repo_url,
        actor.login AS actor_name,
        JSON_EXTRACT_SCALAR(payload,
          "$.push_id") AS push_id,
        JSON_EXTRACT_ARRAY(payload,
            '$.shas') AS array_commits
      FROM
        `githubarchive.day.20130410`
      WHERE
        type='PushEvent'
        AND JSON_EXTRACT(payload,
          "$.shas") IS NOT NULL )))
SELECT
  event_id,
  event_type,
  created_at,
  repo_id,
  repo_name,
  repo_url,
  REGEXP_REPLACE(repo_u

'This query will process 221.54 MB and cost 0.0 USD.'

In [5]:
exporter.execute()

2020-10-11 20:33:28,051 INFO CREATE TABLE: CREATE TABLE IF NOT EXISTS raw_export_test (
                event_id INT64,
                event_type STRING,
                created_at TIMESTAMP,
                repo_id INT64,
                repo_name STRING,
                repo_url STRING,
                repo_fullname STRING,
                push_id INT64,
                actor_name STRING,
                commit_data STRING,
                commit_sha STRING,
                email STRING,
                email_domain STRING,
                email_domain_cleaned STRING)
2020-10-11 20:33:30,723 INFO WRITE_TRUNCATE to codersrank.raw_export_test.raw_export_test


'New table created'

32